In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import os 
import sys 

sys.path.append("../..")
import circletrack_neural as ctn
import plotting_functions as pf
import pca_ica as ica

In [ ]:
## Set parameters
dpath = os.path.abspath('../../CircleTrack/MultiCon_AfterHours/MultiCon_EEG1/post_data/results/mc_EEG1_01/2022_10_19/minian')
sampling_rate = 1/15

In [ ]:
## Load data
spikes = ctn.open_minian(dpath)['S']
pre_spikes = spikes[:, spikes.recording_type.values == 'pre']
session_spikes = spikes[:, spikes.recording_type.values == 'session']
post_spikes = spikes[:, spikes.recording_type.values == 'post']

In [ ]:
## Cell 0's pre recording activity
cell_num = 0
pre_time = np.arange(0, pre_spikes.shape[1] * sampling_rate, sampling_rate)
fig = pf.custom_graph_template(title='Pre Activity', x_title='Time (s)', y_title='Amplitude')
fig.add_trace(go.Scatter(x=pre_time, y=pre_spikes.values[pre_spikes.unit_id == cell_num][0], line_color='grey'))
fig.show()

In [ ]:
## Cell 0's circle track activity
session_time = np.arange(0, session_spikes.shape[1] * sampling_rate, sampling_rate)
fig = pf.custom_graph_template(title='Session Activity', x_title='Time (s)', y_title='Amplitude')
fig.add_trace(go.Scatter(x=session_time, y=session_spikes.values[session_spikes.unit_id == cell_num][0]))
fig.show()

In [ ]:
## Cell 0's post recording activity
post_time = np.arange(0, post_spikes.shape[1], sampling_rate)
fig = pf.custom_graph_template(title='Post Activity', x_title='Time (s)', y_title='Amplitude')
fig.add_trace(go.Scatter(x=post_time, y=post_spikes.values[post_spikes.unit_id == cell_num][0]))
fig.show()

In [ ]:
## Cell 0's entire activity
time = np.arange(0, spikes.shape[1] * sampling_rate, sampling_rate)
fig = pf.custom_graph_template(title='All Activity', x_title='Time (s)', y_title='Amplitude')
fig.add_trace(go.Scatter(x=time, y=spikes.values[spikes.unit_id == cell_num][0], line_color='darkgrey'))
fig.add_vline(x=time[len(pre_time)], line_width=1, line_dash='dash', line_color='turquoise', opacity=1)
fig.add_vline(x=time[len(pre_time) + len(session_time)], line_width=1, line_dash='dash', line_color='turquoise', opacity=1)
fig.show()

In [ ]:
bins = ica.make_bins(pre_spikes.values, 2 * 15)
binned = np.split(pre_spikes.values, bins, axis=1)
len(binned)

In [ ]:
num_spikes = [np.count_nonzero(bin, axis=1) for bin in binned]
num_spikes[26] / 2

In [ ]:
np.vstack(num_spikes).T.shape

In [ ]:
test = ica.bin_transients(pre_spikes.values, bin_size_in_seconds=1, fps=15, analysis_type='num_spikes')
test

In [ ]:
test.values

In [ ]:
## Calcium event rate calculations
def calculate_event_rates(data, bin_size_sec, fps=15, zscore=False):
    if zscore:
        activity = zscore(data, axis=1) ## z-score each cell
    
    
    